preprocessing

In [19]:

library(scales)
library(broom)
library(pillar)
library(Rcpp)
library(plotly)
#install.packages("scales")
#install.packages("broom", type="binary")
#install.packages("pillar", type="binary")
#install.packages("Rcpp")
#install.packages("plotly")
#install.packages('tidyverse')

library(readxl)
library(tidyr)
library(tidyverse)
library(writexl)
library(dplyr)
library(count)


Warning message:
"package 'tidyverse' was built under R version 4.0.5"


ERROR: Error: package or namespace load failed for 'tidyverse' in loadNamespace(j <- i[[1L]], c(lib.loc, .libPaths()), versionCheck = vI[[j]]):
 namespace 'tidyr' 1.1.2 is already loaded, but >= 1.1.3 is required


# Merging explatory variables

In [14]:
# Read files
df <- read_excel("../../data/tidy/Cumulative_Data.xlsx",sheet=1)
df.wb = read_excel("../../data/tidy/wb-data.xlsx",sheet=1)
df.rain = read_excel("../../data/tidy/wb-rain.xlsx",sheet=1)
df.cotwo = read_excel("../../data/tidy/wb-co2-2014.xlsx",sheet=1)
df.fw = read_excel("../../data/tidy/wb-fw-2017.xlsx",sheet=1)
df.wb.mort.gas = read_excel("../../data/tidy/wb-mort-gasprice-2016.xlsx",sheet=1)

# merge files
df.exp <- merge(df.fw,df.cotwo, by="Country Name",all.x=T)
df.exp <- merge(df.exp,df.wb.mort.gas, by="Country Name",all.x=T)
df.exp <- df.exp[,c(1:3,8,10,12)]
df.exp <- merge(df.exp,df.rain, by="Country Name",all.x=T)
df.exp <- df.exp[,c(1:6,8)]


#rename variable title to be short
df.exp <- df.exp %>% 
  rename(
    Country = "Country Name",
      prec = "Average precipitation in depth (mm per year)",
      rifr = "2017 [YR2017] - Renewable internal freshwater resources per capita (cubic meters) [ER.H2O.INTR.PC]",
      cotw = "CO2 emissions from transport (% of total fuel combustion) [EN.CO2.TRAN.ZS] - 2014 [YR2014]",
      moru = "Mortality rate attributed to unsafe water, unsafe sanitation and lack of hygiene (per 100,000 population) [SH.STA.WASH.P5] - 2016 [YR2016]",
      ppfg = "Pump price for gasoline (US$ per liter) [EP.PMP.SGAS.CD] - 2016 [YR2016]"
       )
# count
count(df.exp)
count(df.wb)

#Investiagate NAs
#df.exp$cotw <- as.numeric(df.exp$cotw)
#df.exp[is.na(df.exp)] <- 0
#df.exp[,c(3:7)][is.na(df.exp[,c(3:7)])] <- 0
df.exp[, c(3:7)] <- sapply(df.exp[, c(3:7)], as.numeric)
#df.exp[,3][is.na(df.exp[,3])] <- 0
head(df.exp)

#Renaming the column variables for the world bank data to be consistent with the rest of the data
df.wb <- df.wb %>% 
  rename(
    Year = Time,
    Country = "Country Name",
    cgdp = "GDP (current US$) [NY.GDP.MKTP.CD]",
    tpop = "Population, total [SP.POP.TOTL]",
    upop = "Urban population (% of total population) [SP.URB.TOTL.IN.ZS]",
    popd = "Population density (people per sq. km of land area) [EN.POP.DNST]",
    land ="Land area (sq. km) [AG.LND.TOTL.K2]", 
    lita = "Literacy rate, adult total (% of people ages 15 and above) [SE.ADT.LITR.ZS]",  
    lity = "Literacy rate, youth (ages 15-24), gender parity index (GPI) [SE.ADT.1524.LT.FM.ZS]"  ,
    mori = "Mortality rate, under-5 (per 1,000 live births) [SH.DYN.MORT]" 
       )

# turn character into numbers and numeric
df.wb[, 3] <- sapply(df.wb[, 3], as.numeric)
df.wb[, 5:12] <- sapply(df.wb[, 5:12], as.numeric)

# merge files df.exp and df.wb to world bank source

head(df.wb)

# Clean-up
#Cleaning Column Years
for (i in seq(1, nrow(df))) {
    if (grepl("-", df[i,'Year'])) {
        yy1 = substr(df[i,"Year"], start=1,stop=2)
        yy2 = substr(df[i,"Year"], start=6,stop=7)
        yy3 = paste(yy1,yy2,sep = "")
        #yy3 = as.integer(yy3)
        df[i,"Year"] = yy3
    }
    }

# Finding the Max Years
df$Year = as.numeric(df$Year)
df1 <- matrix(NA, nrow = 0, ncol = length(colnames(df))) #create new empty DF
colnames(df1) = colnames(df) # assign column names to empty data frame
for (country in unique(df$Country)) {
    maxyear = max(df[df$Country==country,'Year'])
    df1 = rbind( df1, df[(df$Country==country) & (df$Year==maxyear),])
} 

#Removing Population Variables (Removal of duplication)
dfh = df1[,!grepl("^P_",names(df1))]

#NA and zeros Analysis
na_count = colSums(is.na(dfh)) #Counting all your NA in each data frames
na_count
na.count.wb = colSums(is.na(df.wb))
na.count.wb

# Removing column for household data frame if NA is greater than 50%`
dfsimple = dfh[, which(colMeans(!is.na(dfh)) > 0.5)]
head(dfsimple)
na_count_cleaned = colSums(is.na(dfsimple)) #Counting all your NA in each data frames
na_count_cleaned
hist(na_count_cleaned)
zeros = colSums(dfsimple != 0) # Counting all your zeros in each data frame

# Understanding wb data and clean up
df.wb <- df.wb[,c(1:2,5:12)]

# # Counting Year less than 2000. 
head(dfsimple$Year , 7)
sum(dfsimple$Year < 2000)
dfsimple[dfsimple$Year < 2000, ]
dfsimplepost2000 = filter(dfsimple, Year >=2000 & Year < 2020)
head(dfsimplepost2000,7)
count(dfsimplepost2000)
write_xlsx(dfsimplepost2000, '../../results/dfsimple.xlsx')

#  additional clean up
setdiff(dfsimplepost2000$Country, df.wb$Country)
# Correct the country names in DF
df.wb [df.wb =='namibia'] = "Namibia"
df.wb [df.wb =='Yemen, Rep.'] = "Yemen" 
df.wb [df.wb =='Gambia, The'] = "Gambia"
df.wb [df.wb =='Egypt, Arab Rep.'] = "Egypt"
df.wb [df.wb =='Congo, Dem. Rep.'] = "Congo Democratic Republic"
df.wb [df.wb =='Congo, Rep.'] = "Congo"
setdiff(dfsimplepost2000$Country, df.wb$Country) # Double Check
df.wb <- merge(x = dfsimplepost2000,
              y = df.wb,
              by = c("Country"))

df.explore <- df.wb[, c(1,33:40 ) ]
setdiff(df.wb$Country, df.exp$Country)
# Correct the country names in DF
df.exp [df.exp =='namibia'] = "Namibia"
df.exp [df.exp =='Yemen, Rep.'] = "Yemen" 
df.exp [df.exp =='Gambia, The'] = "Gambia"
df.exp [df.exp =='Egypt, Arab Rep.'] = "Egypt"
df.exp [df.exp =='Congo, Dem. Rep.'] = "Congo Democratic Republic"
df.exp [df.exp =='Congo, Rep.'] = "Congo"
setdiff(df.wb$Country, df.exp$Country) # Double Check

df.exp <- merge(x = df.explore,
              y = df.exp,
              by = c("Country"))
df.exp <- df.exp[,c(1:9,11:15)]
write_xlsx(df.exp, '../../results/df-wb.xlsx')

ERROR: Error in count(df.exp): could not find function "count"


# Preliminary Data Visualization

In [ ]:
df <- dfsimplepost2000



hist(df$Year, main="",
     xlab="Years",
     ylab="Frequency",
     col.main="red", col.lab="blue")



summary(df$uiws) # House hold with unimproved water source
hist(df$uiws, main="Households using an unimproved water source",
     xlab="Total Percentage",
     ylab="Frequency",
     col.main="red", col.lab="blue")
plot(df$uiws , main="Households using an unimproved water source",
     xlab="Index",
     ylab="Total Percentage",
     col.main="red", col.lab="blue")
summary(df$bicy) # house hold with possession of bicycle. 
hist(df$bicy, main="Households possessing a bicycle",
     xlab="Total Percentage",
     ylab="Frequency",
     col.main="red", col.lab="blue")
plot(df$bicy, main="Households possessing a bicycle",
     xlab="Index",
     ylab="Total Percentage",
     col.main="red", col.lab="blue")
summary(df$Year)
summary(df$Country)

head(df.exp)
summary(df.exp$cgdp) # House hold with unimproved water source
hist(df.exp$cgdp, main="GDP (current US$)",
     xlab="GDP (current US$)",
     ylab="Count ",
     col.main="red", col.lab="blue")
summary(df.exp$cgdp) # House hold with unimproved water source
summary(df.exp)